In [1]:
import sys, os
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import math
# !pip install advent-of-code-data
# https://github.com/wimglenn/advent-of-code-data #for input parsing/loading
from aocd import get_data  # simple, #get_data(day=24, year=2021)
from aocd.models import Puzzle

In [2]:
%matplotlib inline

In [3]:
%load_ext pycodestyle_magic
%pycodestyle_on 
%pycodestyle_off

In [4]:
#  run ONCE
import datetime
today = datetime.datetime.now().day
puzzle = Puzzle(year=2022, day=today)
#print(puzzle.user)
print(puzzle.input_data_fname[60:])
print(puzzle.example_input_data_fname[60:])


iA = puzzle.input_data
iE = puzzle.example_data
print(iE)
print(puzzle.easter_egg)

2022_11_input.txt
2022_11_example_input.txt
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1


AttributeError: 'Puzzle' object has no attribute 'easter_egg'

In [7]:
# test and solve part A
# iA,iE,

def add(a,b):
    return a+b

def mult(a,b):
    return a*b

operation_dict = dict(zip(['+','*'],[add,mult]))
class monkey:
    def __init__(self, lines= None):
        self.nr = -1
        self.items=[]
        self.test_div = -1
        self.operation = dict(zip(['+','*'],[add,mult]))
        self.operation_vals = ['o','o']
        self.throw_to= [-1,-1]
        self.n_inspects = 0
        
        if lines is not None:
            pass
    def __str__(self):
        print()
        print('Print for Monkey #', str(self.nr))
        print('items:',self.items)
        print('div',self.test_div)
        print('operation',self.operation)
        print('throw_to',self.throw_to)
        print('n_inspects',self.n_inspects)
        print()
        return '' 


def parse_to_lists(i=iE):
    lines = i.split('\n')
    index =0
    monkeys = []
    
    while index < len(lines):
        line = lines[index]
        if len(line)>0:
            if line[0]=='M':
                mn  = monkey()
                mn.nr = int(lines[index].split(' ')[-1][:-1])
                mn.items =  [ int(i[:-1]) for i in lines[index +1].split(' ')[4:]]
                mn.items[-1] =  int( lines[index +1].split(' ')[-1][:]) # no comma in last one
                mn.operation =  lines[index +2].split(' ')[4:][2]                
                list_oper_vals= lines[index +2].split(' ')[5:]
                if list_oper_vals[0][0] == 'o': # first
                    mn.operation_vals[0] =list_oper_vals[0][0]
                else:
                    mn.operation_vals[0] = int(list_oper_vals[0])                    
                if list_oper_vals[2][0] == 'o':# second
                    mn.operation_vals[1] = list_oper_vals[2][0]
                else:
                    mn.operation_vals[1] = int(list_oper_vals[2])
                mn.test_div =  int(lines[index +3].split(' ')[5])          
                mn.throw_to[0] =  int(lines[index +4].split(' ')[9])
                mn.throw_to[1] =  int(lines[index +5].split(' ')[9])
                
                monkeys.append(mn)
                
            index = index +7
        else:
            index = index +1
    return monkeys

monkeys=parse_to_lists(iE)
print('nmonks:',len(monkeys))

print(monkeys[1])
#print(monkeys[1].test_div)


nmonks: 4

Print for Monkey # 1
items: [54, 65, 75, 74]
div 19
operation +
throw_to [2, 0]
n_inspects 0




In [52]:

def get_common_mult(monkeys):
    div_=[]
    nr_ = []
    
    for monkey in monkeys:
        div_.append(monkey.test_div)
        nr_.append(monkey.nr)
    
    return np.cumprod(div_)[-1]

def monkey_do(monkeynr,monkeys,worry_level_division=3):
    
    lcm_div = get_common_mult(monkeys)
    nitems = len(monkeys[monkeynr].items)
    #print('items for monkey # ',str(monkeynr) ,' :',monkeys[monkeynr].items)
    for index in range(nitems):
        curr_items= monkeys[monkeynr].items
        worry_level = curr_items[index]
        
        if monkeys[monkeynr].operation_vals[0] == 'o':
            a = worry_level
        else:
            a = monkeys[monkeynr].operation_vals[0]
            
        if monkeys[monkeynr].operation_vals[1] == 'o':
            b = worry_level
        else:
            b = monkeys[monkeynr].operation_vals[1]
            
        #print('ab',a,b)
            
        if monkeys[monkeynr].operation == '*':
            new_worry_level = int(a*b)
            
        elif monkeys[monkeynr].operation == '+':
            
            new_worry_level = int(a+b)
            
        # extra for B, bruteforce way.
        #new_worry_level = np.mod(new_worry_level,lcm_div) # least common multiplier == 9699690 for my example
        new_worry_level= np.floor(new_worry_level/worry_level_division)
        
        
        #print('next worry level',new_worry_level)      
        
        if np.mod(new_worry_level, monkeys[monkeynr].test_div) == 0:
            #new_worry_level = np.mod(new_worry_level,monkeys[monkeys[monkeynr].throw_to[0]].test_div*2) # least common multiplier == 9699690 for my example
            monkeys[monkeys[monkeynr].throw_to[0]].items.append(int(new_worry_level))
            #print('thrown to', str(monkeys[monkeynr].throw_to[0]))
        
        else:
            #new_worry_level = np.mod(new_worry_level,monkeys[monkeys[monkeynr].throw_to[1]].test_div*2)# least common multiplier == 9699690 for my example
            monkeys[monkeys[monkeynr].throw_to[1]].items.append(int(new_worry_level))
            #print('thrown to', str(monkeys[monkeynr].throw_to[1]))
        

        
        monkeys[monkeynr].n_inspects +=1
    
    #clear all items in the cue (thrown)
    #print('Stack processed for Monkey #', str(monkeys[monkeynr].nr))
    monkeys[monkeynr].items = []
    return monkeys

monkeys=parse_to_lists(iE)
print('nmonks:',len(monkeys))
monkeys=monkey_do(0,monkeys)
monkeys=monkey_do(1,monkeys)
monkeys=monkey_do(2,monkeys)





nmonks: 4


In [53]:

def process_round(monkeys,worry_level_division=3):

    for index in range(len(monkeys)):
        monkeys=monkey_do(index,monkeys,worry_level_division=worry_level_division)
    #print('Round processed')
    return(monkeys)

def count_inspects(monkeys):
    inspects_=[]
    nr_ = []
    
    for monkey in monkeys:
        inspects_.append(monkey.n_inspects)
        nr_.append(monkey.nr)
        
    return inspects_,nr_

def calc_monk_business(monkeys):
    inspects_,nr_ = count_inspects(monkeys)
    inspects_.sort(reverse=True)
    return(inspects_[0]*inspects_[1])
#monkeys=monkey_do(0,monkeys)


monkeys=parse_to_lists(iA)
for n in range(20):
    #print('At roudn n:',n)
    #print(monkeys[2].items)
    
    process_round(monkeys,worry_level_division=3)
print(monkeys[2])
inspects_,nr_ = count_inspects(monkeys)
print(inspects_,nr_)
answ_A = calc_monk_business(monkeys)
print(answ_A)

#print(get_common_mult(monkeys))


Print for Monkey # 2
items: [2, 7, 7, 7]
div 5
operation *
throw_to [1, 6]
n_inspects 134


[20, 127, 134, 222, 226, 93, 194, 47] [0, 1, 2, 3, 4, 5, 6, 7]
50172


In [45]:
puzzle.answer_a = answ_A # this submits!

Part a already solved with different answer: 50172


In [32]:

# test and solve part B, use inputs & test inputs.
# for B
monkeys=parse_to_lists(iA)
for n in range(10000):
    #print('At roudn n:',n)
    monkeys=process_round(monkeys,worry_level_division=1)
print(monkeys[2])
inspects_,nr_ = count_inspects(monkeys)
print('inspects',inspects_,nr_)
answ_B = calc_monk_business(monkeys)
print(monkeys[0])

print(answ_B)
#puzzle.answer_b = answ_B # this submits!


Print for Monkey # 2
items: [4200386, 6315356, 9378416, 4605575, 5338900, 6027707, 2622490, 1318174]
div 5
operation *
throw_to [1, 6]
n_inspects 73630


inspects [10659, 70776, 73630, 109098, 106461, 62841, 61442, 50365] [0, 1, 2, 3, 4, 5, 6, 7]

Print for Monkey # 0
items: [762699]
div 17
operation *
throw_to [5, 3]
n_inspects 10659


11614682178


In [21]:
print(answ_B)

117066964


In [33]:
puzzle.answer_b =answ_B

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 11! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
